In [ ]:
%load_ext autoreload
%autoreload 2
import sys, os, time, json, re
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

import data_preprocessing as dp
import backtrader as bt
import matplotlib.pyplot as plt

## Get data

In [ ]:
frequency = timedelta(seconds=60)
pair = 'USDT_BTC'
date_start = '2020-11-11'
date_end = '2021-03-31'
lob_depth = 100
norm_type = 'dyn_z_score'
roll = 1440*10 # 10 days

In [ ]:
df_data, df_data_stdz = dp.import_data(
    pair, 
    date_start, 
    date_end, 
    frequency=frequency, 
    depth=lob_depth, 
    norm_type=norm_type, 
    roll=roll, 
    stdz_depth=1
)

In [ ]:
df_data['Mid_Price'].plot()

## Resample

In [ ]:
df_data[['Mid_Price']].head(31)

In [ ]:
# resample data to a less granular frequency
df_data = df_data.asfreq('1min')
df_data['volume'] = df_data['amount_buy'] + df_data['amount_sell']

data_resampled = df_data.resample('30min', label='right').agg( # closing time of candlestick
    {
    'Mid_Price': ['last', 'first', np.max, np.min], 
    'volume': np.sum
    }
)

data_resampled.columns = data_resampled.columns.get_level_values(1)

data_resampled['close'] = data_resampled['last']
data_resampled['open'] = data_resampled['first']
data_resampled['high'] = data_resampled['amax']
data_resampled['low'] = data_resampled['amin']
data_resampled['volume'] = data_resampled['sum']
data_resampled.index.name = 'datetime'

data_resampled
# rename columns

In [ ]:
data_resampled['log_ret'] = (np.log(data_resampled['close']) - np.log(data_resampled['close'].shift(1)))
data_resampled['roll_std'] = data_resampled['log_ret'].rolling(window=336).std() # 336 is the number of 30mins interval in week
data_resampled['roll_std'].plot(figsize=(8,4))

## Backtrader

In [ ]:
from Strategies.GoldenCross import GoldenCross
from Strategies.BuyHold import BuyHold

# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(GoldenCross)

# Create a Data Feed
data = bt.feeds.PandasData(dataname=data_resampled[:2000])

# Add the Data Feed to Cerebro
cerebro.adddata(data)

cerebro.addwriter(bt.WriterFile, out='./Strategies/logging/golden_cross2.csv', csv=True)

# Set our desired cash start
cerebro.broker.setcash(200000.0)
# Add a FixedSize sizer according to the stake
# cerebro.addsizer(bt.sizers.PercentSizer, percents=10)
# cerebro.broker.setcommission(commission=0.0007) 

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything


cerebro.run()

plt.rcParams['figure.figsize']=[22, 16]
cerebro.plot()
# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

# figure out what's wrong with stop losses


In [ ]:
strategy_results = pd.read_csv('./Strategies/logging/golden_cross2.csv', header=1, index_col='Id').dropna(thresh=3)
strategy_results['datetime'] = pd.to_datetime(strategy_results['datetime'])
print(strategy_results.shape)

In [ ]:
strategy_and_indic = pd.merge(data_resampled, strategy_results, left_index=True, right_on='datetime', how='outer')
print(strategy_and_indic.columns)
columns_to_keep = ['datetime', 'open_x', 'close_x', 'high_x', 'low_x', 'cash', 'value', 'buy', 'sell', 'pnlplus', 'pnlminus', 'sma', 'sma.1', 'crossover']
strategy_and_indic[columns_to_keep].to_csv('./Strategies/logging/golden_cross_cl.csv')

In [ ]:
# def saveplots(cerebro, numfigs=1, iplot=True, start=None, end=None,
#              width=16, height=9, dpi=300, tight=True, use=None, file_path = '', **kwargs):

#         from backtrader import plot
#         if cerebro.p.oldsync:
#             plotter = plot.Plot_OldSync(**kwargs)
#         else:
#             plotter = plot.Plot(**kwargs)

#         figs = []
#         for stratlist in cerebro.runstrats:
#             for si, strat in enumerate(stratlist):
#                 rfig = plotter.plot(strat, figid=si * 100,
#                                     numfigs=numfigs, iplot=iplot,
#                                     start=start, end=end, use=use)
#                 figs.append(rfig)

#         for fig in figs:
#             for f in fig:
#                 f.savefig(file_path, bbox_inches='tight')
#         # return figs

# saveplots(cerebro, file_path = 'savefig.png') 

## My trading functions

In [ ]:
## Roadmap
# for each trade I need entry price, closing price, number of periods, time in the trade, min, max, volatility V 
# make execution assumptions: conservative: enter trade next open bar, exit trade next open bar V
# add stops and trailing stops
# pull more data
# add single strategy to binance account with cctx
# backtest multiple strategies across multiple pairs
# deploy multiple strategies

In [ ]:
import ccxt
import ta
from ta.volatility import BollingerBands, AverageTrueRange
from ta.trend import EMAIndicator
import config
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
## Exchange connectivity
exchange = ccxt.binance(
    {
        'apiKey': config.BINANCE_API_KEY,
        'secret': config.BINANCE_SECRET_KEY
    }
)

markets = exchange.load_markets()

bars = exchange.fetch_ohlcv('ETH/USDT', limit=20) # most recent candle keeps evolving

In [ ]:
df =data_resampled[['open', 'high', 'low', 'close', 'volume']].copy() # pd.DataFrame(bars, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])

## Create Indicators
# Bollinger Bands
bb_indicator = BollingerBands(df['close'], window=20)

df['upper_band'] = bb_indicator.bollinger_hband()
df['lower_band'] = bb_indicator.bollinger_lband()
df['moving_average'] = bb_indicator.bollinger_mavg()

# Average True Range
atr_indicator = AverageTrueRange(df['high'], df['low'], df['close'])
df['atr'] = atr_indicator.average_true_range()

# Moving Averages
ema50_indicator = EMAIndicator(df['close'], 50)
df['ema_50'] = ema50_indicator.ema_indicator()

ema20_indicator = EMAIndicator(df['close'], 20)
df['ema_20'] = ema20_indicator.ema_indicator()

In [ ]:
## Generate Signals
# EMA cross
df['ema_cross_signal'] = np.where(
    df['ema_20'] > df['ema_50'], 1, 
    np.where(df['ema_20'] < df['ema_50'], -1, 0))

df['ema_cross_trades'] = np.where(
    df['ema_cross'].diff() > 0, 'buy', 
    np.where(df['ema_cross'].diff() < 0, 'sell', 'hold'))

df['ema_cross_position'] = np.where(
    df['ema_cross'].diff() > 0, +1, 
    np.where(df['ema_cross'].diff() < 0, -1, 0))

In [ ]:
## Backtesting
initial_cash = 1000

df['returns'] = np.log(df['close']) - np.log(df['close'].shift(1))
df['ema_cross_returns'] = df['returns'] * df['ema_cross_signal']

df['ema_cross_cum_performance'] = np.exp(df['ema_cross_returns'].cumsum())
df['ema_cross_cash'] = df['ema_cross_cum_performance'] * initial_cash

np.exp(df['returns'].cumsum()).plot(figsize=(8,4), legend=True) # reverse log returns to prices
df['ema_cross_cum_performance'].plot(legend=True)

In [458]:
# prepare df trades

# get positions in the dataframe where indicator generates signals
open_trades_idx = np.where(df['ema_cross_position']!=0)[0]
# -2 because of shape is n rows and df is 0 indexed and because we do + 1 later - avoid out of bound error
closing_trades_idx = np.append(open_trades_idx, df.shape[0]-2)[1:] 
df_trades = df.iloc[open_trades_idx][['ema_cross_position']].copy() # empty dataframe with only datetime index

# entry and closing points
df_trades['entry_price'] = df.iloc[open_trades_idx+1]['open'].values # assume entry trade is executed at the next bar open
df_trades['closing_price'] = df.iloc[closing_trades_idx+1]['open'].values # assume closing is executed at the next bar open

# trade discrete returns
df_trades['discrete_return'] = (df_trades['closing_price'] / df_trades['entry_price']) - 1

# how long are the trades 
df_trades['trade_n_periods'] = closing_trades_idx - open_trades_idx
df_trades['trade_duration'] = df.iloc[closing_trades_idx].index - df.iloc[open_trades_idx].index

# what happened throughout the trade
df['trade_grouper'] = np.nan
df.loc[df.iloc[open_trades_idx].index, 'trade_grouper'] = df.iloc[open_trades_idx].index
df['trade_grouper'] = df['trade_grouper'].fillna(method='ffill')
df.head(60)

all_trades_list = []
for name, sub_df in df.groupby(by='trade_grouper'):
    max_val = sub_df['high'].max()
    min_val = sub_df['low'].min()
    returns_std = sub_df['returns'].std()

    all_trades_list.append([name, max_val, min_val, returns_std])


intra_trade_stats = pd.DataFrame(all_trades_list, columns=['datetime', 'px_high', 'px_low', 'returns_std']).set_index('datetime')
df_trades = pd.merge(df_trades, intra_trade_stats, left_index=True, right_index=True)


def max_dd(row):
    ''' Measure of how "painful" holding the trade was '''
    if row['ema_cross_position'] == 1:
        return row['entry_price'] - row['px_low']
    elif row['ema_cross_position'] == -1:
        return - (row['entry_price'] - row['px_high'])
    else:
        return 0

df_trades['dd'] = df_trades.apply(max_dd, axis=1)

# calculate trade returns and jump into risk management / stop losses


In [464]:
df_trades.apply(lambda x: (x['closing_price'] / x['entry_price']) - 1)

KeyError: 'closing_price'

In [460]:
## Metrics
# Net Profit
net_profit = df['ema_cross_cash'][-1] - initial_cash 

# Max Drowdown
max_dd = df_trades['dd'].max()

# Win Ratio


In [461]:
net_profit, max_dd

(4434.277784299962, 3073.0216667346685)

In [463]:
df_trades#[trade_life_metrics]


,ema_cross_position,entry_price,closing_price,trade_n_periods,trade_duration,px_high,px_low,returns_std,dd
datetime,,,,,,,,,
2020-11-12 01:00:00,1,15602.266253,16172.253856,106,2 days 05:00:00,16459.977145,15577.496531,0.003723,24.769723
2020-11-14 06:00:00,-1,16172.253856,16117.802589,54,1 days 03:00:00,16200.401434,15728.018881,0.003627,28.147579
2020-11-15 09:00:00,1,16117.802589,15882.015012,18,0 days 09:00:00,16148.513662,15964.215493,0.001961,153.587095
2020-11-15 18:00:00,-1,15882.015012,16133.933620,23,0 days 11:30:00,16041.572305,15791.911717,0.002677,159.557293
2020-11-16 05:30:00,1,16133.933620,17525.977876,151,3 days 03:30:00,18446.418405,16054.259355,0.005904,79.674265
...,...,...,...,...,...,...,...,...,...
2021-03-26 08:00:00,1,53517.653382,55172.378570,121,2 days 12:30:00,56523.180023,52597.728468,0.004490,919.924914
2021-03-28 20:30:00,-1,55172.378570,57240.218968,25,0 days 12:30:00,56259.611864,54833.853787,0.004037,1087.233294
2021-03-29 09:00:00,1,57240.218968,57431.408258,99,2 days 01:30:00,59754.819768,56184.584676,0.004346,1055.634292


In [437]:
type(df[[]])

pandas.core.frame.DataFrame

In [350]:
closing_trades_idx - open_trades_idx

array([106,  54,  18,  23, 151,  10, 121,  38,   7,  15,  19,  31,  73,
       123, 155,  37,   9,   8,  78,  56,  45,   8,  50,  23,   6,  80,
        10,  85, 453,  55,  64,  41, 205,  40, 261,  64, 232,  86,  15,
        42,  98,  47,  15,  54,  10,  12,  76, 130,  39,  35,  21,  21,
        43, 110,  89,  38,   8,  58, 181,   9,  91,  56, 110,  42,  94,
        31,  49,  29,  58,   1, 104,  12, 135, 101,  25,  13,  17,   3,
        17,  67,   6,  82,  82,  16,  55,  75,  33,  19,  72,   7, 136,
         6,  50,   7,  20,   7,  76,   6,   1,   9,   1,  83,  14,   2,
         6,  20,  58,  14,   1,   1,  80,  62,   3,   3,   7,  86,  25,
        69, 121,  25,  99,  10,  16])

In [340]:
open_trades_idx+1

array([  50,  156,  210,  228,  251,  402,  412,  533,  571,  578,  593,
        612,  643,  716,  839,  994, 1031, 1040, 1048, 1126, 1182, 1227,
       1235, 1285, 1308, 1314, 1394, 1404, 1489, 1942, 1997, 2061, 2102,
       2307, 2347, 2608, 2672, 2904, 2990, 3005, 3047, 3145, 3192, 3207,
       3261, 3271, 3283, 3359, 3489, 3528, 3563, 3584, 3605, 3648, 3758,
       3847, 3885, 3893, 3951, 4132, 4141, 4232, 4288, 4398, 4440, 4534,
       4565, 4614, 4643, 4701, 4702, 4806, 4818, 4953, 5054, 5079, 5092,
       5109, 5112, 5129, 5196, 5202, 5284, 5366, 5382, 5437, 5512, 5545,
       5564, 5636, 5643, 5779, 5785, 5835, 5842, 5862, 5869, 5945, 5951,
       5952, 5961, 5962, 6045, 6059, 6061, 6067, 6087, 6145, 6159, 6160,
       6161, 6241, 6303, 6306, 6309, 6316, 6402, 6427, 6496, 6617, 6642,
       6741, 6751])

In [341]:
closing_trades_idx+1

array([ 156,  210,  228,  251,  402,  412,  533,  571,  578,  593,  612,
        643,  716,  839,  994, 1031, 1040, 1048, 1126, 1182, 1227, 1235,
       1285, 1308, 1314, 1394, 1404, 1489, 1942, 1997, 2061, 2102, 2307,
       2347, 2608, 2672, 2904, 2990, 3005, 3047, 3145, 3192, 3207, 3261,
       3271, 3283, 3359, 3489, 3528, 3563, 3584, 3605, 3648, 3758, 3847,
       3885, 3893, 3951, 4132, 4141, 4232, 4288, 4398, 4440, 4534, 4565,
       4614, 4643, 4701, 4702, 4806, 4818, 4953, 5054, 5079, 5092, 5109,
       5112, 5129, 5196, 5202, 5284, 5366, 5382, 5437, 5512, 5545, 5564,
       5636, 5643, 5779, 5785, 5835, 5842, 5862, 5869, 5945, 5951, 5952,
       5961, 5962, 6045, 6059, 6061, 6067, 6087, 6145, 6159, 6160, 6161,
       6241, 6303, 6306, 6309, 6316, 6402, 6427, 6496, 6617, 6642, 6741,
       6751, 6767])

In [ ]:
# # df['close'].plot(legend=True)
# ((np.exp(df['ema_cross_returns'].cumsum()) * 100)).plot(legend=True)
# # ((np.exp(df['returns'].cumsum()) * df['close'][0])).plot(legend=True)

In [ ]:
# df['ema_cross_position'].cumsum().plot()
# df['ema_cross_signal'].plot()

In [ ]:
## Plotting
plot_indic_list = ['ema_50', 'ema_20']
plot_indic_color = ['#CCFFFF', '#FFCCFF']

fig = make_subplots(
    rows=2, 
    cols=1,
    shared_xaxes=True,
    row_heights=[0.2, 0.8],
    vertical_spacing=0.02
)

fig.add_trace(
    go.Candlestick(
        x=df.index,
        open=df['open'],
        high=df['high'],
        low=df['low'],
        close=df['close'],
        name='px',
        increasing_line_color= 'green', 
        decreasing_line_color= 'red'
    ),
    row=2, 
    col=1
)
# candlestick xaxes
fig.update_xaxes(rangeslider_visible=False,    row=2,
    col=1)


# add indicators to candlestick chart
for indic, color in zip (plot_indic_list, plot_indic_color):
    fig.add_scatter(
        x=df.index, 
        y=df[indic], 
        name=indic, 
        marker=dict(color=color),
        row=2, 
        col=1
    )

# add buy trades marks
fig.add_scatter(
    x=df.index, 
    y=df['close']+100, 
    showlegend=False,
    # name='trades', 
    mode='markers',
    marker=dict(
        size=12,
        # I want the color to be green if 
        # lower_limit ≤ y ≤ upper_limit
        # else red
        color=(
            (df['ema_cross_trades'] == 'buy')).astype('int'),
        colorscale=[[0, 'rgba(255, 0, 0, 0)'], [1, '#B7FFA1']],
        symbol=5
    ),
    row=2, 
    col=1
)

# add sell trades marks
fig.add_scatter(
    x=df.index, 
    y=df['close']-100, 
    showlegend=False,
    # name='trades', 
    mode='markers',
    marker=dict(
        size=12,
        # I want the color to be green if 
        # lower_limit ≤ y ≤ upper_limit
        # else red
        color=(
            (df['ema_cross_trades'] == 'sell')).astype('int'),
        colorscale=[[0, 'rgba(255, 0, 0, 0)'], [1, '#FF7F7F']],
        symbol=6   
        ),
        row=2, 
        col=1
)

# add strategy returns
fig.add_scatter(
    x=df.index,
    y=df['ema_cross_cum_performance'],
    name='cum_performance',
    row=1,
    col=1
)

# general layout
fig.update_layout(
    width=1400,
    height=600,
    title='<b>Strategy</b>',
    title_x=.5,
    yaxis_title='USDT/BTC',
    template="plotly_dark",
    # plot_bgcolor='rgb(10,10,10)'
)

fig.show()